In [6]:
from Utils.MidiParseHandler import *
from pathlib import Path
import glob
import os
import music21 as m21

In [7]:
midi_files = glob.glob(os.path.join('Dataset-RAW', '*.mid'))

In [8]:
import concurrent.futures
import multiprocessing
import Utils  # Import the module where multi_readed is defined
import os
import shutil

# Assuming you have a list of MIDI file paths
progress_file = 'progress.txt'
output_folder = 'processed_midi_files'

num_processes = multiprocessing.cpu_count()

# Convert the data using multiprocessing with timeout
converted_data, failed_files = multiprocess_data_conversion(midi_files, num_processes)

# Save or use the converted data
# For example, save to a file or further process the converted data
# Here we just print the number of successfully converted MIDI objects
print(f"Successfully converted {len(converted_data)} MIDI files.")
print(f"Failed to convert {len(failed_files)} MIDI files.")
print(f"Failed files: {failed_files}")

all_midis = converted_data

Processed 1/152 files
Processed 2/152 files
Processed 3/152 files
Processed 4/152 files
Processed 5/152 files
Processed 6/152 files
Processed 7/152 files
Processed 8/152 files
Processed 9/152 files
Processed 10/152 files
Processed 11/152 files
Processed 12/152 files
Processed 13/152 files
Processed 14/152 files
Processed 15/152 files
Processed 16/152 files
Processed 17/152 files
Processed 18/152 files
Processed 19/152 files
Processed 20/152 files
Processed 21/152 files
Processed 22/152 files
Processed 23/152 files
Processed 24/152 files
Processed 25/152 files
Processed 26/152 files
Processed 27/152 files
Processed 28/152 files
Processed 29/152 files
Processed 30/152 files
Processed 31/152 files
Processed 32/152 files
Processed 33/152 files
Processed 34/152 files
Processed 35/152 files
Processed 36/152 files
Processed 37/152 files
Processed 38/152 files
Processed 39/152 files
Processed 40/152 files
Processed 41/152 files
Processed 42/152 files
Processed 43/152 files
Processed 44/152 fil

In [30]:
all_midis[0]

MidiFile(type=1, ticks_per_beat=480, tracks=[
  MidiTrack([
    MetaMessage('time_signature', numerator=3, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('key_signature', key='Bb', time=0),
    MetaMessage('set_tempo', tempo=326087, time=0),
    Message('control_change', channel=0, control=121, value=0, time=0),
    Message('program_change', channel=0, program=0, time=0),
    Message('control_change', channel=0, control=7, value=100, time=0),
    Message('control_change', channel=0, control=10, value=64, time=0),
    Message('control_change', channel=0, control=91, value=30, time=0),
    Message('control_change', channel=0, control=93, value=30, time=0),
    MetaMessage('midi_port', port=0, time=0),
    Message('note_on', channel=0, note=82, velocity=49, time=0),
    Message('note_on', channel=0, note=82, velocity=0, time=479),
    Message('note_on', channel=0, note=81, velocity=49, time=1),
    Message('note_on', channel=0, note=81, velocit

In [9]:
def choose_right_instrument(part):
    cont = 0
    piano1 = []
    piano2 = []
    first = False

    for m in part.tracks:
        for mex in m:
            if ('program_change' in str(mex)):
                #print("Cambio di strumento")
                if not first:
                    first = True
                else:
                    first = False
            if ('note_on' in str(mex)):
                if first:
                    piano1.append(str(mex))
                    #print("aggiungo al primo piano")
                else:
                    piano2.append(str(mex))
                    #print("aggiungo al secondo piano")

    return piano1, piano2

In [10]:
import re

def extract_note_number_with_regex(s):
    match = re.search(r'note=(\d+)', s)
    if match:
        return int(match.group(1))
    return None  # In case there's no 'note=' pattern found


In [11]:
def extract_notes_and_chords(stream):
    chord = []
    stream_on = []
    for mex in stream:
        if ('note_on' in str(mex)):
            if (not 'velocity=0' in str(mex)):
                #print(mex)
                m = extract_note_number_with_regex(mex)
                chord.append(m)
            else:
                #print(chord)
                if(len(chord) > 0):
                    #print(chord)
                    stream_on.append(chord)
                    chord = []
    return stream_on

In [12]:
def extract_both_stream(stream1, stream2):
    part1 = extract_notes_and_chords(stream1)
    filtered_elements1 = [el for el in part1 if el not in ([], '', {}, ())]
    part2 = extract_notes_and_chords(stream2)
    filtered_elements2 = [el for el in part2 if el not in ([], '', {}, ())]
    return filtered_elements1,filtered_elements2

In [13]:
def parse_corpus(all_midis):
    source = []
    for midi in all_midis:
        piano1, piano2 = choose_right_instrument(midi)
        source.append(extract_both_stream(piano1, piano2))
    return source

In [14]:
parsed = parse_corpus(all_midis)

In [15]:
def map_notes_and_chords_unified(note_list, map, next_id):
    unified_map = map

    for notes in note_list:
        key = tuple(sorted(notes))  # Use a tuple of notes sorted as the key
        if key not in unified_map:
            unified_map[key] = next_id
            next_id += 1

    return unified_map, next_id

# Example use case
notes_and_chords = [[60], [60, 64, 67], [62], [62, 65, 69], [60]]
unified_ids, _ = map_notes_and_chords_unified(notes_and_chords, {}, 0)
print("Unified Map:", unified_ids)

Unified Map: {(60,): 0, (60, 64, 67): 1, (62,): 2, (62, 65, 69): 3}


In [16]:
def map_nested_notes_and_chords_unified(data):
    unified_map = {}
    next_id = 1

    for stream1, stream2 in data:  # each element contains two lists
        map1, next_id = map_notes_and_chords_unified(stream1, unified_map, next_id)
        unified_map.update(map1)
        map2, next_id = map_notes_and_chords_unified(stream2, unified_map, next_id)
        unified_map.update(map2)

    return unified_map

mapp = map_nested_notes_and_chords_unified(parsed)

In [31]:
len(mapp)

12346

In [32]:
mapp

{(82,): 1,
 (81,): 2,
 (79,): 3,
 (77,): 4,
 (75,): 5,
 (84,): 6,
 (75, 79): 7,
 (77, 82): 8,
 (79, 82): 9,
 (80, 84): 10,
 (81, 84): 11,
 (78, 81): 12,
 (86,): 13,
 (74,): 14,
 (74, 86): 15,
 (81, 82): 16,
 (72,): 17,
 (70,): 18,
 (67,): 19,
 (78, 86): 20,
 (79, 83, 86): 21,
 (69,): 22,
 (65,): 23,
 (76,): 24,
 (78,): 25,
 (62,): 26,
 (63,): 27,
 (58,): 28,
 (60,): 29,
 (57,): 30,
 (55,): 31,
 (58, 62): 32,
 (57, 60): 33,
 (55, 58): 34,
 (55, 58, 60): 35,
 (55, 58, 62): 36,
 (55, 58, 62, 65): 37,
 (55, 60, 63, 67): 38,
 (53, 57, 62, 65): 39,
 (55, 58, 62, 67): 40,
 (60, 63, 67): 41,
 (58, 62, 65): 42,
 (55, 58, 63): 43,
 (56, 60, 63): 44,
 (53, 57, 60): 45,
 (53,): 46,
 (51, 55, 58): 47,
 (50,): 48,
 (52,): 49,
 (54,): 50,
 (62, 67, 70): 51,
 (62, 69, 70): 52,
 (63, 67, 70): 53,
 (60, 62, 65, 69): 54,
 (53, 58, 62, 65): 55,
 (55, 58, 63, 67): 56,
 (58, 70): 57,
 (57, 69): 58,
 (53, 65): 59,
 (55, 58, 65): 60,
 (53, 58, 62): 61,
 (31, 43): 62,
 (29, 41): 63,
 (27, 39): 64,
 (26, 38): 6

In [17]:
next_id = len(mapp.keys())

In [22]:
def apply_mapping(nested_list, mapping_dict, next_id):
    # Applying the mapping to each element in each sublist of each list
    # return [[[mapping_dict.get(item, item) for item in inner_list] for inner_list in sublist] for sublist in nested_list]
    container = []
    song = []
    piano1 = []
    piano2 = []

    cont = 0

    for sublist in nested_list:
        for stream in sublist:
            cont += 1
            for note in stream:
                if (cont % 2 == 0):
                    note = tuple(note)
                    try:
                        piano1.append(mapping_dict[note])
                    except:
                        mapping_dict[note] = next_id
                        next_id += 1
                        piano1.append(mapping_dict[note])
                else:
                    note = tuple(note)
                    try:
                        piano2.append(mapping_dict[note])
                    except:
                        mapping_dict[note] = next_id
                        next_id += 1
                        piano2.append(mapping_dict[note])
            song.append(piano1)
            song.append(piano2)
        container.append(song)
    return container

    

In [23]:
final = apply_mapping(parsed, mapp, next_id)

In [26]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

from torch.nn.utils.rnn import pad_sequence

class MIDIDataset(Dataset):
    def __init__(self, input_sequences, targets):
        # Convert each sequence to a tensor when initializing the dataset
        self.inputs = [torch.tensor(sequence, dtype=torch.float32) for sequence in input_sequences]
        self.targets = [torch.tensor(target[0], dtype=torch.long) for target in targets]  # Assuming target is a single note index

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]
    

def collate_fn(batch):
    # Separate sequences and targets from the batch
    sequences, targets = zip(*batch)
    # Pad sequences dynamically
    sequences_padded = pad_sequence([torch.tensor(seq) for seq in sequences], batch_first=True, padding_value=0)
    targets = torch.tensor(targets)
    return sequences_padded, targets


# Example usage
dataset = MIDIDataset(final[0], final[1])
# Assuming your dataset is defined properly
data_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

In [27]:
import torch
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


completamente sbagliato qui sto ritornando una sequenza non una nota singola

In [29]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
import random
import matplotlib.pyplot as plt

device = torch.device("mps")

class Seq2Seq(nn.Module):
	# device definition that depends on the gpu availability
		def __init__(self, input_dim, rnn_dim=512, rnn_layers=2, thr=0):
	# input_dim is the pianoroll size
				super(Seq2Seq, self).__init__()

				self.thr=thr
				self.input_dim=input_dim
				self.rnn_dim=rnn_dim
				self.rnn_layers=rnn_layers

				# encoder decoder definiction based on LSTM with dropout network
				self.encoder = nn.LSTM( input_size=input_dim, hidden_size=rnn_dim, num_layers=rnn_layers, batch_first=True, dropout=0.2)
				self.decoder = nn.LSTM( input_size=input_dim, hidden_size=rnn_dim, num_layers=rnn_layers, batch_first=True, dropout=0.2)
				# the clasification is based on two fully conected networks dropout 
				self.classifier = nn.Sequential(
						nn.Linear(rnn_dim, 256),
						nn.ReLU(),
						nn.Dropout(0.5),
						nn.Linear(256, input_dim)
				)

				# loss function
				self.loss_function = nn.BCEWithLogitsLoss() #combines logsoftmax with NLLLoss

			# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
		def forward(self, x,y,teacher_forcing_ratio = 0.5):          
				# Train method with teacher forcing ratio to introduce more or less real notes in the training step

				# zero padding in the x for the encoder.
				x0 = torch.zeros(x.shape[0], 1, x.shape[2]).to(device)
				x = torch.cat([x0,x], dim=1)

				output, (hn, cn) = self.encoder(x)

				seq_len = y.shape[1]

				# initializing the output to zeros.
				outputs = torch.zeros(y.shape[0], seq_len, self.input_dim).to(device)

				# zero padding in the y for the decoder
				y0 = torch.zeros(y.shape[0], 1, y.shape[2]).to(device)
				y = torch.cat([y0,y], dim=1)

				# initializing the output to zeros.
				input = y[:,0,:].view(y.shape[0],1,y.shape[2])

				# we generate iteratively the future time steps selecting th real o predicted note based on a random scheme 
				# conditioned to teacher_forcing_ratio value: 1 is 100% teacher forcing, 0 only predicted notes
				for t in range(1, seq_len):
						output, (hn, cn) = self.decoder(input, (hn, cn))

						teacher_force = random.random() < teacher_forcing_ratio   # teacher forcing draw

						shape = output.shape

						# we add one dimension to classify all bacthes at same time
						x=output.unsqueeze(2)

						x = self.classifier(x) #softmax is'nt necessary

						# removing the extra dimension
						x = x.view(shape[0],shape[1],-1)

						# applying the threshold to the predicted value
						output = (x > self.thr).float()

						# we choose the predicted or real note for the next iteration based on teacher forcing random draw
						input = (y[:,t,:].view(y.shape[0],1,y.shape[2]) if teacher_force else output.view(y.shape[0],1,y.shape[2]))

						# we accumulate the new note
						outputs[:,t,:] = x.view(y.shape[0],-1)

				return outputs

In [ ]:

68 0 0 0 0 0 - 0.15s
68 0 0 0 0 0 
68 69 0 0 0 0
